# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 12:40PM,261418,10,Eme-112602,Emerginnova,Released
1,Apr 19 2023 12:38PM,261417,10,131136,"Virtue Rx, LLC",Released
2,Apr 19 2023 12:38PM,261417,10,9396013,"Virtue Rx, LLC",Released
3,Apr 19 2023 12:34PM,261416,15,7830834,"Brookfield Pharmaceuticals, LLC",Released
4,Apr 19 2023 12:01PM,261415,19,ELI04202023,Eli Lilly and Company,Released
5,Apr 19 2023 11:58AM,261414,19,TR-RIH-04202023,"GCH Granules USA, Inc.",Released
6,Apr 19 2023 11:08AM,261406,20,TR-ROM-040002023,Granules Consumer Health,Executing
7,Apr 19 2023 11:07AM,261404,15,9386994,"Brookfield Pharmaceuticals, LLC",Executing
8,Apr 19 2023 11:07AM,261404,15,9387613,"Brookfield Pharmaceuticals, LLC",Executing
9,Apr 19 2023 11:07AM,261404,15,9387625,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,261414,Released,1
28,261415,Released,1
29,261416,Released,1
30,261417,Released,2
31,261418,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261414,NaN,NaN,1.0
261415,NaN,NaN,1.0
261416,NaN,NaN,1.0
261417,NaN,NaN,2.0
261418,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260869,0.0,7.0,26.0
260910,0.0,9.0,4.0
261117,0.0,9.0,1.0
261321,0.0,13.0,12.0
261343,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260869,0,7,26
260910,0,9,4
261117,0,9,1
261321,0,13,12
261343,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260869,0,7,26
1,260910,0,9,4
2,261117,0,9,1
3,261321,0,13,12
4,261343,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260869,,7,26
1,260910,,9,4
2,261117,,9,1
3,261321,,13,12
4,261343,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 12:40PM,261418,10,Emerginnova
1,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC"
3,Apr 19 2023 12:34PM,261416,15,"Brookfield Pharmaceuticals, LLC"
4,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company
5,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc."
6,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health
7,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC"
12,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC"
17,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC"
19,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 19 2023 12:40PM,261418,10,Emerginnova,,,1
1,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",,,2
2,Apr 19 2023 12:34PM,261416,15,"Brookfield Pharmaceuticals, LLC",,,1
3,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company,,,1
4,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc.",,,1
5,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,,1,
6,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC",,4,1
7,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC",,4,1
8,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC",,2,
9,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC,,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:40PM,261418,10,Emerginnova,1,,
1,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2,,
2,Apr 19 2023 12:34PM,261416,15,"Brookfield Pharmaceuticals, LLC",1,,
3,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company,1,,
4,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc.",1,,
5,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,,1,
6,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC",1,4,
7,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC",1,4,
8,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC",,2,
9,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC,9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:40PM,261418,10,Emerginnova,1,,
1,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2,,
2,Apr 19 2023 12:34PM,261416,15,"Brookfield Pharmaceuticals, LLC",1,,
3,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company,1,,
4,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:40PM,261418,10,Emerginnova,1.0,NaN,NaN
1,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2.0,NaN,NaN
2,Apr 19 2023 12:34PM,261416,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
3,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company,1.0,NaN,NaN
4,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:40PM,261418,10,Emerginnova,1.0,0.0,0.0
1,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2.0,0.0,0.0
2,Apr 19 2023 12:34PM,261416,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
3,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company,1.0,0.0,0.0
4,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1045589,4.0,1.0,0.0
15,1568310,29.0,30.0,7.0
19,1829752,26.0,22.0,1.0
20,1305712,40.0,26.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1045589,4.0,1.0,0.0
1,15,1568310,29.0,30.0,7.0
2,19,1829752,26.0,22.0,1.0
3,20,1305712,40.0,26.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,4.0,1.0,0.0
1,15,29.0,30.0,7.0
2,19,26.0,22.0,1.0
3,20,40.0,26.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,4.0
1,15,Released,29.0
2,19,Released,26.0
3,20,Released,40.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,7.0,1.0,0.0
Executing,1.0,30.0,22.0,26.0
Released,4.0,29.0,26.0,40.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,7.0,1.0,0.0
1,Executing,1.0,30.0,22.0,26.0
2,Released,4.0,29.0,26.0,40.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,7.0,1.0,0.0
1,Executing,1.0,30.0,22.0,26.0
2,Released,4.0,29.0,26.0,40.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()